In [1]:
import json
import numpy as np
from collections import defaultdict
import openai
import re
import os
from openai import OpenAI

In [2]:
def save_json(data, filepath=r'new_data.json'):
    with open(filepath, 'w') as fp:
        json.dump(data, fp, indent=4)

In [3]:
client=OpenAI()
def call_gpt(client, messages, user, model='gpt-4o-mini', format="json",temp=0.7):
    if format == "json":
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            response_format={ "type": "json_object" },
            temperature=temp,
            user = user,
            logprobs=True
        )
        response_dict = dict(response.choices[0].message)
        logprobs = [token.logprob for token in response.choices[0].logprobs.content]
        response_dict["perplexity_score"] = np.exp(-np.mean(logprobs))
        if user!=None:
            response_dict['user']=user
            response_dict['role'] = "user"
            del response_dict['function_call']
            del response_dict['tool_calls']
            return response_dict,json.loads(response.choices[0].message.content)
        else:
            return response_dict, json.loads(response.choices[0].message.content)
    else:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0.7,
            user = user,
            logprobs=True
        )
        return response_dict,response.choices[0].message.content

different simulations for different eval questions, try different responses. Make student prompts more random and simple and less restrictive. Only specify in prompt to answer an eval question.

Teacher class containing all teacher prompts. General response prompts used for now

In [4]:
class Teacher:
    def __init__(self,personas) :
        self.personas=personas

    def lecture(self,actions):
        schema = {"lecture":"lecture"}
        messages = {
                "role": "system", 
                "content": f"""
                    You are a high school teacher who has to teach students having different ethnic diversities and personality traits. 
                    The profiles of the students is given in {self.personas}. Some student actions classified into good and bad actions is given in: {actions}.
                    Correct the students if a bad action is performed and praise them for performing a good action. 
                    The more times a bad action is repeated the more strict the reply should be.  
                    The topic of today's class is 'Climate Change and Global Warming'. 
                    You need to create a small lecture on the topic. Take into consideration the backgrounds of the students but do not specifically address any students.
                    The lecture should explain the topic well and should engage different kinds of students. 
                    The summary should be human-readable.
                    Strictly reply in JSON format which follows the schema: {schema}
                """
            }
        return messages

    # def lecture(self):
    #     schema = {"summary":"summary", "questions":{"Q1":"Question 1", "Q2":"Question 2"}}
    #     messages = {
    #             "role": "system", 
    #             "content": f"""
    #                 You are a high school teacher who has to teach students from different ethnic diversities. 
    #                 The ethnic profiles of the students is given in {self.personas}.
    #                 Your abilities as a teacher are giving lectures, asking questions specific to a student's background and answer questions asked by students.
    #                 The topic of today's class is 'Climate Change and Global Warming'. 
    #                 You need to create a small lecture on the topic. Take into consideration the backgrounds of the students but do not specifically address any students.
    #                 The lecture should explain the topic well. 
    #                 Reply by giving a summary of the lecture and a set of questions to test the knowledge of the students.
    #                 Consider the student profiles while crafting the questions but they should be answerable by all students. 
    #                 The questions can be scenario based.
    #                 Strictly reply with a single summary and two questions. 
    #                 The summary and questions should be human-readable.
    #                 Strictly reply in JSON format which follows the schema: {schema}
    #             """
    #         }
        
    #     # response_msg,response = call_gpt(client, messages,user=None, format="json") 
    #     return messages
    def general_response(self):
        messages = {
                "role": "system", 
                "content": f"""
                    You need to respond to the students who have certain comments/questions about the lecture. 
                    The response should either be a command, request, question, correction or general statement depending on the students response.
                    You are allowed to b have negative comments if the student is taking conversations off-topic or distracting the class. 
                    Penalize behaviour that is not common by replying in a negative manner.
                    Highly consider the students background information while responding.   
                    The response should be concise and human-readable.
                    Strictly reply in JSON format which follows the schema
                """
            }
        return messages
    def answer(self,questions,persona):
        schema = {"A1":"answer 1", "A2":"answer 2"}
        messages = {
                "role": "system", 
                "content": f"""
                    The student:{persona} has now answered your questions and asked further questions given in {questions}.
                    Answer these questions very concisely by strictly considering their background profile.
                    Strictly answer in two sentences or less.
                    The answer should be human-readable.
                    Strictly reply in JSON format which follows the schema: {schema}
                """
            }
        # response_msg,response = call_gpt(client, messages,user=None, format="json") 
        return messages
    
    
    def evaluate(self,questions):
        schema = {"Question 1":{"question":"question","dialogues":"dialogues by students that are relevant for answering the question.","answer":"answer","learnings":"What did you learn from the incident"},}
        messages ={
                "role": "system", 
                "content": f"""
                    The students have now given a summary of their learnings and feedback for the lecture.
                    Now you have to assess your teaching experience based on the past conversations with the students by answering the questions in {questions}.
                    Answer the questions strictly based on how the students responded during the lecture, their learnings and their feedback. 
                    The answer should highlight dialogues from different students relevant to the question, how you handled the situation and what you learnt.
                    Answer each question very concisely.
                    Strictly Reply in JSON format: {schema}.                
                """
            }
        return messages


Baseline evaluation prompt

In [5]:
def evaluate_baseline(questions):
    schema = [{"Question 1":{"question":"question","answer":"answer"},}]
    messages = [{
            "role": "system", 
            "content": f"""
                You are a high school teacher who has to teach students having varied ethnic diversities and personality traits.
                You have to answer these questions: {questions} to demonstrate your experience as a teacher.
                The answers should be concise and human-readable. 
                Strictly Reply in JSON format with schema: {schema}.
            """
        }]
    
    response_msg,response = call_gpt(client, messages,user=None, format="json",temp=0.0) 
    return response_msg,response

Student class containing all student prompts. General response and student summary prompts used currently.

In [6]:
class Student():
    def __init__(self,persona, user_id):
        self.persona=persona
        self.user_id = user_id

    def general_response(self,persona,conversation):
        messages = [{
                "role": "system", 
                "content": f"""
                    The past interraction in the lecture is given in {conversation}.
                    You need to respond to the teacher's comments/question. 
                    Highly consider your background profile given in {persona}, while responding.
                    Your response should strictly be one of the actions mentioned in your profile.  
                    The response should be concise and human-readable.
                """
            }]
        response_msg,response = call_gpt(client, messages,user=self.user_id, format="json") 
        return response_msg, response
    
    def student_QA(self,summary,questions):
        schema = {"Answers":{"A1":"answer 1", "A2":"answer 2"}, "Questions":{"Q1" : "question 1","Q2" : "question 2"}}
        messages = [ 
                {
                    "role": "system", 
                    "content": f"""
                        You are a high school student whose profile is given by {self.persona}. 
                        The topic of today's class is 'Climate Change and Global Warming'.
                        The teacher will provide the summary of the lecture and some questions for you to answer.
                        You need to answer the questions and ask further questions strictly according to the background information given in your profile. 
                        The answers and new questions can be scenario based or a personal experience unique to your background. 
                        Strictly ask two or lesser questions.
                        The answers and the questions should be very concise and human-readable.
                        Strictly reply in JSON format which follows the schema: {schema}
                    """
                },
                { "role": "user", "content": f"This is the summary of the lecture:{summary}"},
                { "role": "user", "content": f"These are the questions you need to answer:{questions}"}
        ]
        response_msg,response = call_gpt(client, messages,user=self.user_id, format="json") 
        return response_msg, response
    
    def answer(self,summary,questions):
        schema =  {"A1":"answer 1", "A2":"answer 2"}
        messages = [ 
                {
                    "role": "system", 
                    "content": f"""
                        You are a high school student whose profile is given by {self.persona}. 
                        The topic of today's class is 'Climate Change and Global Warming'.
                        The teacher will provide the summary of the lecture and some questions for you to answer.
                        You need to answer the questions these questions by strictly considering your background details. 
                        The answers can be scenario based on a personal experience unique to your background. 
                        The answers should be very concise and human-readable.
                        Strictly reply in JSON format which follows the schema: {schema}
                    """
                },
                { "role": "user", "content": f"This is the summary of the lecture:{summary}"},
                { "role": "user", "content": f"These are the questions you need to answer:{questions}"}
        ]
        response_msg,response = call_gpt(client, messages,user=self.user_id, format="json") 
        return response_msg, response
    
    def student_summ(self,persona,conversation):
        schema = {"feedback":"student feedback"}
        messages = [ 
                {
                    "role": "system", 
                    "content": f"""
                        You are a high school student whose profile is given by {persona}. 
                        The topic of today's class is 'Climate Change and Global Warming'.
                        The past interraction in the class is given in {conversation}.
                        You need to give feedback to the teacher strictly based on your past interraction with the teacher and your background details.
                        The feedback can include suggestions, what you liked?, what you disliked?, were your queries answered by the teacher?, what improvements should you make as a student?, what improvements the teacher should make?, etc. 
                        The feedback should strictly consider your profile details and past interraction with the teacher.
                        Strictly reply in JSON format which follows the schema: {schema}
                    """
                }
        ]
        response_msg,response = call_gpt(client, messages,user=self.user_id, format="json") 
        return response_msg, response

In [7]:
conversations = []

Helper functions. Not used currently

In [8]:
def preprocess_lecture(lecture):
    split = lecture.split("[")
    summary=split[0]
    questions = split[1:]
    return summary,questions

def preprocess_student_qa(student_qa):
    split = student_qa.split(",[")
    answers=split[0]
    questions = split[1:]
    return questions,answers

def preprocess_student_qa1(student_qa):
    split = student_qa.split("Q1")
    answers=split[0]
    questions = split[1:]
    return questions,answers

def questions_dict(questions):
    questions_list = []
    for i,question in enumerate(questions):
        temp = {}
        temp['user'] = "student {id}".format(id=i+1)
        temp['questions'] = question
        questions_list.append(temp)
    return questions_list

In [9]:
personas = json.load(open(r'data\personas.json'))
questions_eval = json.load(open(r'data\questions.json'))
actions = json.load(open(r'data\Actions_cat.json'))
# content = json.load(open(r'C:\Projects\LLM_experiments\content.json'))

teacher = Teacher(personas)
student_1 = Student(personas[0],"Amina Hussein")
student_2 = Student(personas[1], "Rajesh Kumar")
student_3 = Student(personas[2],"Svetlana Petrov")

In [10]:
personas[2]

{'Name': 'Svetlana Petrov',
 'user': 'student 3',
 'Origin': 'Moscow, Russia',
 'Background': 'Svetlina is a creative and imaginative person. She likes to joke around. She is a part of various clubs. She presents solutions to problems in a very creative way.',
 'Actions': {'Action 1': 'Make a joke.'}}

Sample simulation: Lecture -> General student responses -> General teacher responses -> student summary and feedback -> evaluate

In [11]:
messages = []
with open('data\Conversations\conv.txt', 'w') as f:
    # Teacher delivers lecture
    messages.append(teacher.lecture(actions))
    res,lecture = call_gpt(client,messages=messages,user=None, format="json",temp=0.7)
    messages.append(res)
    f.write("Teacher:\t"+str(lecture)+"\n")

    # Student 1 general response and teachers response to the student
    messages.append(teacher.general_response())
    res_msg,res = student_1.general_response(personas[0],messages)
    messages.append(res_msg)
    f.write("Student_1: \t"+ str(res)+"\n")
    res_msg,res = call_gpt(client,messages=messages,user=None, format="json",temp=0.7)
    messages.append(res_msg)
    f.write("Teacher:\t"+str(res)+"\n")

    # Student 2 general response(object) and teachers response to the student
    res_msg,res = student_2.general_response(personas[1],messages)
    messages.append(res_msg)
    f.write("Student_2: \t"+ str(res)+"\n")
    res_msg,res = call_gpt(client,messages=messages,user=None, format="json",temp=0.7)
    messages.append(res_msg)
    f.write("Teacher:\t"+str(res)+"\n")

    # Student 2 repeated objection and teachers response to the student
    res_msg,res = student_2.general_response(personas[1],messages)
    messages.append(res_msg)
    f.write("Student_2: \t"+ str(res)+"\n")
    res_msg,res = call_gpt(client,messages=messages,user=None, format="json",temp=0.7)
    messages.append(res_msg)
    f.write("Teacher:\t"+str(res)+"\n")

    # Student 3 general response(joke) and teachers response to the student
    res_msg,res = student_3.general_response(personas[2],messages)
    messages.append(res_msg)
    f.write("Student_3: \t"+ str(res)+"\n")
    res_msg,res = call_gpt(client,messages=messages,user=None, format="json",temp=0.7)
    messages.append(res_msg)
    f.write("Teacher:\t"+str(res)+"\n")

     # Student 2 repeated joke and teachers response to the student
    res_msg,res = student_3.general_response(personas[2],messages)
    messages.append(res_msg)
    f.write("Student_3: \t"+ str(res)+"\n")
    res_msg,res = call_gpt(client,messages=messages,user=None, format="json",temp=0.7)
    messages.append(res_msg)
    f.write("Teacher:\t"+str(res)+"\n")

    # Students summary and feedback
    res_msg,res = student_1.student_summ(personas[0],messages)
    messages.append(res_msg)
    f.write("Student_1: \t"+ str(res)+"\n")

    res_msg,res = student_2.student_summ(personas[1],messages)
    messages.append(res_msg)
    f.write("Student_2: \t"+ str(res)+"\n")

    res_msg,res = student_3.student_summ(personas[2],messages)
    messages.append(res_msg)
    f.write("Student_3: \t"+ str(res)+"\n")

    # Teacher evaluate
    messages.append(teacher.evaluate([questions_eval["Question 2"],questions_eval["Question 15"],questions_eval["Question 7"],questions_eval["Question 10"],questions_eval["Question 5"],questions_eval["Question 8"]]))
    res_msg,res = call_gpt(client,messages=messages,user=None, format="json",temp=0.0)
    messages.append(res_msg)
    messages.append(res)
    f.write("Teacher: \t"+ str(res)+"\n")
    
    f.close()
save_json(messages,r"C:\Projects\LLM_experiments\data\Conversations\conversations_test.json")

<>:2: SyntaxWarning: invalid escape sequence '\C'
<>:2: SyntaxWarning: invalid escape sequence '\C'
C:\Users\HP\AppData\Local\Temp\ipykernel_19976\540994345.py:2: SyntaxWarning: invalid escape sequence '\C'
  with open('data\Conversations\conv.txt', 'w') as f:


Baseline evaluation

In [211]:
questions_eval_baseline = json.load(open(r'data\questions_baseline.json'))
res_msg,res = evaluate_baseline([questions_eval["Question 2"],questions_eval["Question 15"],questions_eval["Question 7"],questions_eval["Question 10"],questions_eval["Question 5"],questions_eval["Question 8"]])
save_json(res,r"C:\Projects\LLM_experiments\data\Conversations\conversations_base_test.json")

In [181]:
questions_eval_baseline = json.load(open(r'data\questions_baseline.json'))
res_msg,res = evaluate_baseline(questions_eval_baseline)
save_json(res,r"C:\Projects\LLM_experiments\data\Conversations\conversations_base.json")

AttributeError: 'Teacher' object has no attribute 'evaluate_baseline'

In [ ]:
# questions = questions_dict([qa_1['Questions'],qa_2['Questions'],qa_3['Questions']])

    # res_msg, res = teacher.answer(conversations,questions)
    # conversations.append([res_msg])
    # f.write("Teacher: \t"+ str(res))

    # f.write("Student_3: \t"+ str(qa))
    # print(student_1_qa)
    # questions_1,answers_1 = preprocess_student_qa1(student_1_qa)
    # f.write("Student_1: \t Here are the answers to your questions\t" + str(answers_1) +"I have these further questions:\t"+str(questions_1)+ '\n\n')
    # student_2_qa = student_2.student_QA(summ,questions)
    # questions_2,answers_2 = preprocess_student_qa1(student_2_qa)
    # f.write("Rajesh: \t Here are the answers to your questions\t" + str(answers_2) +"I have these further questions:\t"+str(questions_2)+ '\n\n')
    # student_3_qa = student_3.student_QA(summ,questions)
    # print(student_3_qa)
    # questions_3,answers_3 = preprocess_student_qa1(student_3_qa)
    # f.write("Svetlana: \t Here are the answers to your questions\t" + str(answers_3) +"I have these further questions:\t"+str(questions_3)+ '\n\n')
    # teacher_answers_1 = teacher.answer(f,questions_1,personas[0])
    # f.write("Teacher: \t Here are the answers to your questions " + str(teacher_answers_1) + '\n\n')
    # teacher_answers_2 = teacher.answer(f,questions_2,personas[1])
    # f.write("Teacher: \t Here are the answers to your questions " + str(teacher_answers_2) + '\n\n')
    # teacher_answers_3 = teacher.answer(f,questions_3,personas[2])
    # f.write("Teacher: \t Here are the answers to your questions " + str(teacher_answers_3) + '\n\n')
    # feedback_1 = student_1.student_summ(f,personas[0])
    # feedback_1,summary_1 = preprocess_student_qa(feedback_1)
    # f.write("Amina: \t This is the summary of my learnings " + str(summary_1) + "This is my feedback for the lecture:"+str(feedback_1)+'\n\n')
    # feedback_2 = student_2.student_summ(f,personas[1])
    # feedback_2,summary_2 = preprocess_student_qa(feedback_2)
    # f.write("Rajesh: \t This is the summary of my learnings " + str(summary_2) + "This is my feedback for the lecture:"+str(feedback_2)+'\n\n')
    # feedback_3 = student_3.student_summ(f,personas[2])
    # feedback_3,summary_3 = preprocess_student_qa(feedback_3)
    # f.write("Svetlana: \t This is the summary of my learnings " + str(summary_3) + "This is my feedback for the lecture:"+str(feedback_3)+'\n\n')
    # evaluation = teacher.evaluate(f)
    # f.write("Teacher:\tAssessment report for each student: " + str(evaluation) +'\n\n')